In [1]:
import pandas as pd
import numpy as np
import os


import sklearn
import matplotlib.pyplot as plt

print('Library imported.')

Library imported.


In [5]:
names = ["Length", "Diameter", "Height", "Whole weight",
	"Shucked weight", "Viscera weight", "Shell weight", "Age"]
df = pd.read_csv('abalone.data', sep=',', names = names)

df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [6]:
from sklearn.model_selection import train_test_split

df_x = df[df.columns[:-1]]
df_y = df['Age']


x_train, x_valid, y_train, y_valid = train_test_split(df_x, df_y, test_size = 0.2, random_state = 9)

In [16]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_scaler = scaler.fit(x_train)
x_train_pre = x_scaler.transform(x_train)
x_test_pre = x_scaler.transform(x_valid)
y_scaler = scaler.fit(y_train.values.reshape(-1, 1))
y_train_pre = y_scaler.transform(y_train.values.reshape(-1, 1))
y_test_pre = y_scaler.transform(y_valid.values.reshape(-1, 1))

In [17]:
from sklearn.svm import SVR




model = SVR()
model.fit(x_train_pre, y_train_pre)

print('Normal R2 score without hyper param tuning: ', model.score(x_test_pre, y_test_pre))

C:\Users\lawrence123\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Normal R2 score without hyper param tuning:  0.5021719408681944


In [18]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

kernel = ["linear", "rbf", "sigmoid", "poly"]
tolerance = [1e-3, 1e-4, 1e-5, 1e-6]
C = [1, 1.5, 2, 2.5, 3]

grid = dict(kernel=kernel, tol=tolerance, C=C)

In [19]:
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
gridSearch = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1,
	cv=cvFold, scoring="neg_mean_squared_error")

searchResults = gridSearch.fit(x_train_pre, y_train_pre)

# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(x_test_pre, y_test_pre)))

[INFO] grid searching over the hyperparameters...


C:\Users\lawrence123\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[INFO] evaluating...
R2: 0.51


In [20]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

kernel = ["linear", "rbf", "sigmoid", "poly"]
tolerance = loguniform(1e-6, 1e-3)
C = [1, 1.5, 2, 2.5, 3]

grid = dict(kernel=kernel, tol=tolerance, C=C)

In [23]:
print("[INFO] grid searching over the hyperparameters...")
cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
gridSearch = RandomizedSearchCV(estimator=model, param_distributions=grid, n_jobs=-1,
	cv=cvFold, scoring="neg_mean_squared_error")

searchResults = gridSearch.fit(x_train_pre, y_train_pre)

# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(x_test_pre, y_test_pre)))

[INFO] grid searching over the hyperparameters...


C:\Users\lawrence123\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[INFO] evaluating...
R2: 0.51


### GridSearch and RandomizedSearch got identical model performance, but randomizedsearch could achieve by smaller time cost. For multiple parameters, RandomizedSearch is preferred over gridsearch.